# Object Detection

* [SSD-Mobilenet]
* [SSD-Resnet50]


[Docker Containers of the network](https://github.com/ctuning/ck-object-detection/tree/master/docker)

## Table of Contents

1. [Overview](#overview)
1. [Platform](#platform)
1. [Experimental data](#data) [for developers]
1. [Data wrangling code](#code) [for developers]
1. [Experiments on Hikey](#experiments_hikey)
   1. [TensorFlow](#experiments_tensorflow_hikey)
1. [Experiments on Firefly](#experiments_firefly)
   1. [TensorFlow](#experiments_tensorflow_firefly)

<a id="overview"></a>
## Overview

This Jupyter Notebook studies performance (execution time) vs accuracy (mAP and Recall) of different Object Detection networks, on different size objects (large, medium and small).
Moreover the experiments are performed on different architectures, in particular CPU and GPU, to evaluate the benefit of GPU acceleration.

<a id="platform"></a>
## Platform

<a id="CPU info"></a>
### CPU

  - Model:
    - Intel Xeon
  - Version:
    - E5-2650 v3;  
  - Frequency:
    - 2.30GHz;
  - Number of Cores (physical):
    - 10
  - HyperThreading
    - Yes


  - RAM:
    - 32 GB;

  - BSP:
    - Ubuntu 16.04 LTS Linux


<a id="GPU info"></a>
### GPU

  - Model:
    - Nvidia GeForce GTX 1080
  - Frequency:
    - 1.6GHz;
  - CUDA Version:
    - 10.2
  - Driver Version:
    - 430.14
  - MEMORY:
    - 8 GB;


<a id="data"></a>
## Get the experimental data

<a id="code"></a>
## Data wrangling code

**NB:** Please ignore this section if you are not interested in re-running or modifying this notebook.

### Includes

#### Standard

In [ ]:
import os
import sys
import json
import re

#### Scientific

If some of the scientific packages are missing, please install them using:
```
# pip install jupyter pandas numpy matplotlib
```

In [ ]:
import IPython as ip
import pandas as pd
import numpy as np
import matplotlib as mp
import seaborn as sb

In [ ]:
print ('IPython version: %s' % ip.__version__)
print ('Pandas version: %s' % pd.__version__)
print ('NumPy version: %s' % np.__version__)
print ('Matplotlib version: %s' % mp.__version__)
print ('Seaborn version: %s' % sb.__version__)

In [ ]:
from IPython.display import Image, display
def display_in_full(df):
    pd.options.display.max_columns = len(df.columns)
    pd.options.display.max_rows = len(df.index)
    display(df)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

In [ ]:
default_colormap = cm.autumn
default_fontsize = 16
default_barwidth = 0.8
default_figwidth = 24
default_figheight = 3
default_figdpi = 200
default_figsize = [default_figwidth, default_figheight]

In [ ]:
if mp.__version__[0]=='2': mp.style.use('classic')
mp.rcParams['figure.max_open_warning'] = 200
mp.rcParams['figure.dpi'] = default_figdpi
mp.rcParams['font.size'] = default_fontsize
mp.rcParams['legend.fontsize'] = 'medium'

#### Collective Knowledge

If CK is not installed, please install it using:
```
# pip install ck
```

If data are not present, download and add the repository to ck using:
```
!wget https://www.dropbox.com/s/532l5yb1qvxp8q6/my_experiments_0.zip?dl=0

!mv my_experiments_0.zip\?dl\=0 my_experiments_0.zip
!ck add repo:my-repo-with-experiments --zip=my_experiments_0.zip
```

In [ ]:
import ck.kernel as ck
print ('CK version: %s' % ck.__version__)

!wget https://www.dropbox.com/s/532l5yb1qvxp8q6/my_experiments_0.zip?dl=0

!mv my_experiments_0.zip\?dl\=0 my_experiments_0.zip
!ck add repo:my_repo_with_experiments --zip=my_experiments_0.zip

In [ ]:
import ck.kernel as ck
print ('CK version: %s' % ck.__version__)


    
repo_uoa = 'my_repo_with_experiments'
!ck list $repo_uoa:experiment:* | sort

### Access experimental data

In [ ]:
def get_experimental_results(repo_uoa, tags='', accuracy=True,
                             module_uoa='experiment', _library=None, _platform=None):
    r = ck.access({'action':'search', 'repo_uoa':repo_uoa, 'module_uoa':module_uoa, 'tags':tags})
    from pprint import pprint
    pprint (r)
    if r['return']>0:
        print('Error: %s' % r['error'])
        exit(1)
    experiments = r['lst']

    dfs = []
    for experiment in experiments:
        data_uoa = experiment['data_uoa']
        r = ck.access({'action':'list_points', 'repo_uoa':repo_uoa, 'module_uoa':module_uoa, 'data_uoa':data_uoa})
        pprint(r)
        if r['return']>0:
            print('Error: %s' % r['error'])
            exit(1)
        # Mapping of expected library tags to reader-friendly names.
        tag_to_name = {
            # ArmCL tags on HiKey.
            '17.12-48bc34ea'    : 'armcl-17.12',
            '18.01-f45d5a9b'    : 'armcl-18.01',
            '18.03-e40997bb'    : 'armcl-18.03',
            'request-d8f69c13'  : 'armcl-dv/dt', # armcl-18.03+
            '18.05-b3a371bc'    : 'armcl-18.05',
            # ArmCL tags on Firefly.
            '17.12-48bc34e'     : 'armcl-17.12',
            '18.01-f45d5a9'     : 'armcl-18.01',
            '18.03-e40997b'     : 'armcl-18.03',
            '18.05-b3a371b'     : 'armcl-18.05',
            # TensorFlow tags.
            'tensorflow-1.7'    : 'tensorflow-1.7',
            'tensorflow-1.8'    : 'tensorflow-1.8',
        }
            
        # Library.
        library_tags = [ tag for tag in r['dict']['tags'] if tag in tag_to_name.keys() ]
        #if len(library_tags)==1:
        #    library = tag_to_name[library_tags[0]]
        #else:
        #    print('[Warning] Bad library tags. Skipping experiment with tags:')
        #    print(r['dict']['tags'])
        #    continue
        #if _library and _library!=library: continue
        # For each point.   
        
        pipeline_file_path = os.path.join(r['path'], 'pipeline.json')
        with open(pipeline_file_path) as pipeline_file:
            pipeline_data_raw = json.load(pipeline_file)
        weights_env = pipeline_data_raw['dependencies']['weights']['dict']['env']
        pprint(weights_env)
        
        tags = r['dict']['tags']
        print (tags)
        for point in r['points']:
            point_file_path = os.path.join(r['path'], 'ckp-%s.0001.json' % point)
            with open(point_file_path) as point_file:
                point_data_raw = json.load(point_file)
            characteristics_list = point_data_raw['characteristics_list']

            pprint(characteristics_list)
            print ("****************************")
            print (point_data_raw.keys())
            print ("****************************")
            
            pprint (point_data_raw['choices']['env'])
            num_repetitions = len(characteristics_list)
            platform = point_data_raw['features']['platform']['platform']['model']
            #if _platform and _platform!=platform: continue
            batch_size = np.int64(point_data_raw['choices']['env'].get('CK_BATCH_SIZE',-1))
            batch_count = np.int64(point_data_raw['choices']['env'].get('CK_BATCH_COUNT',-1))
            characteristics = characteristics_list[0]
            #convolution_method = convolution_method_to_name[np.int64(point_data_raw['choices']['env'].get('CK_CONVOLUTION_METHOD_HINT',1))]
            #if library.startswith('tensorflow-'):
            #    multiplier = np.float64(point_data_raw['choices']['env'].get('CK_ENV_TENSORFLOW_MODEL_MOBILENET_MULTIPLIER',-1))
            #    resolution = np.int64(point_data_raw['choices']['env'].get('CK_ENV_TENSORFLOW_MODEL_MOBILENET_RESOLUTION',-1))
            #else:
            #    multiplier = np.float64(point_data_raw['choices']['env'].get('CK_ENV_MOBILENET_WIDTH_MULTIPLIER',-1))
            #    resolution = np.int64(point_data_raw['choices']['env'].get('CK_ENV_MOBILENET_RESOLUTION',-1))
            #model = 'v1-%.2f-%d' % (multiplier, resolution)
            if accuracy:
                data = [
                    {
                        # features
                        #'platform': platform,
                        # choices
                        #'weights_env': weights_env['CK_ENV_TENSORFLOW_MODEL_FROZEN_GRAPH'],
                        'model': tags[0],
                        'tf_version':tags[1],
                        'batch_size': batch_size,
                        'batch_count': batch_count,
                        # runtime characteristics
                        'mAP': characteristics['run'].get('mAP', 0)*100,
                        'Recall': characteristics['run'].get('recall', 0)*100,
                        'mAP_large': characteristics['run']['metrics'].get('DetectionBoxes_Recall/AR@100 (large)', 0)*100,
                        'mAP_medium': characteristics['run']['metrics'].get('DetectionBoxes_Recall/AR@100 (medium)', 0)*100,
                        'mAP_small': characteristics['run']['metrics'].get('DetectionBoxes_Recall/AR@100 (small)', 0)*100,
#                         # recompute accuracy from frame_predictions (was incorrectly recorded in early experiments)
#                         'accuracy_top1_': len([
#                             prediction for prediction in characteristics['run'].get('frame_predictions', [])
#                             if prediction['accuracy_top1']=='yes'
#                         ]) / np.float64(batch_count),
#                         'accuracy_top5_': len([
#                             prediction for prediction in characteristics['run'].get('frame_predictions', [])
#                             if prediction['accuracy_top5']=='yes'
#                         ]) / np.float64(batch_count)
                    }
#                    for (repetition_id, characteristics) in zip(range(num_repetitions), characteristics_list)
                ]
            else: # performance
                data = [
                    {
                        # features
                        #'platform': platform,
                        # choices
                        #'weights_env': weights_env['CK_ENV_TENSORFLOW_MODEL_FROZEN_GRAPH'], 
                        'model': tags[0],
                        'tf_version':tags[1],
                        'batch_size': batch_size,
                        'batch_count': batch_count,
                        # statistical repetition
                        'repetition_id': repetition_id,
                        # runtime characteristics
                        'avg_fps': characteristics['run'].get('avg_fps', 'n/a'),
                        'avg_time_ms': characteristics['run']['avg_time_ms'],
                        'graph_load_time_ms': characteristics['run']['graph_load_time_s']*1e+3,
                        'images_load_time_avg_ms': characteristics['run']['images_load_time_avg_s']*1e+3,
                    }
                    for (repetition_id, characteristics) in zip(range(num_repetitions), characteristics_list)
                ]
            index = [
                'model', 'tf_version', 'batch_size', 'batch_count'
                #, 'mAP', 'Recall', 'mAP_large', 'mAP_medium', 'mAP_small'
            ]
            # Construct a DataFrame.
            df = pd.DataFrame(data)
            df = df.set_index(index)
            # Append to the list of similarly constructed DataFrames.
            dfs.append(df)
    if dfs:
        # Concatenate all thus constructed DataFrames (i.e. stack on top of each other).
        result = pd.concat(dfs)
        result.sort_index(ascending=True, inplace=True)
    else:
        # Construct a dummy DataFrame the success status of which can be safely checked.
        result = pd.DataFrame(columns=['success?'])
    return result
!ck recache repo
dfs = get_experimental_results(repo_uoa)

dfs_per = get_experimental_results(repo_uoa,accuracy=False)

### Plot experimental data

#### Plot accuracy (bar plot)

In [ ]:
def plot_accuracy(df_raw, groupby_level='batch_size', performance_metric=['mAP','mAP_large','mAP_medium','mAP_small'], title=None, figsize=None, rot=90):
    from pprint import pprint
    pprint(df_raw[performance_metric].values)
    pprint(df_raw.index.values)
    df_bar = pd.DataFrame(
        data=df_raw[performance_metric].values, columns=performance_metric,
        index=pd.MultiIndex.from_tuples(
            tuples=[ (m,l,s,c) for (m,l,s,c) in df_raw.index.values ],
            names=[ 'model', 'tf_version', 'batch_size', 'batch_count' ]
        )
    )
    #groupby decide quanti plot fare.
    #unstack come separare le colonne (?)
    unstack_level = 'model'
    colormap = cm.autumn
    xlabel='(tf_version, batch_size)'
    #df_bar.columns.names = ['time']
    #if groupby_level=='convolution_method':
    #    unstack_level = 'library'
    #    xlabel='(Model [channel multiplier - input resolution], Convolution Method)'
    #    colormap = cm.autumn
    #elif groupby_level=='library':
    #    unstack_level = 'convolution_method'
    #    xlabel='(Library, Model [channel multiplier - input resolution])'
    #    colormap = cm.summer
    # Set default style.
    ylabel='mAP %'
    if not title: title = 'prova' 
    if not figsize: figsize = [default_figwidth, 8]
    pprint (df_bar.index.names[:-1])
    # Plot 
    mean = df_bar.groupby(level=df_bar.index.names[:-1]).mean().unstack(unstack_level)
    std = df_bar.groupby(level=df_bar.index.names[:-1]).std().unstack(unstack_level)
    axes = mean.groupby(level=groupby_level) \
        .plot(yerr=std, kind='bar', grid=True, width=0.8, rot=rot, figsize=figsize,
              fontsize=default_fontsize, colormap=colormap)
    for ax in axes:
        # Title.
        ax.set_title(title)
        # X label.
        ax.set_xlabel(xlabel)
        # Y axis.
        ax.set_ylabel(ylabel)
plot_accuracy(dfs, rot=0)

#### Plot performance (bar plot)

In [ ]:
def plot_performance(df_raw, groupby_level='batch_size', performance_metric=['avg_fps','avg_time_ms','graph_load_time_ms','images_load_time_avg_ms'], title=None, figsize=None, rot=90):
    from pprint import pprint
    pprint(df_raw[performance_metric].values)
    pprint(df_raw.index.values)
    df_bar = pd.DataFrame(
        data=df_raw[performance_metric].values, columns=performance_metric,
        index=pd.MultiIndex.from_tuples(
            tuples=[ (p,s,c,w) for (p,w,s,c) in df_raw.index.values ],
            names=[ 'model', 'batch_size', 'batch_count', 'tf_version' ]
        )
    )
    #groupby decide quanti plot fare.
    #unstack come separare le colonne (?)
    unstack_level = 'model'
    colormap = cm.autumn
    xlabel='(batch_count, tf_version)'
    #df_bar.columns.names = ['time']
    #if groupby_level=='convolution_method':
    #    unstack_level = 'library'
    #    xlabel='(Model [channel multiplier - input resolution], Convolution Method)'
    #    colormap = cm.autumn
    #elif groupby_level=='library':
    #    unstack_level = 'convolution_method'
    #    xlabel='(Library, Model [channel multiplier - input resolution])'
    #    colormap = cm.summer
    # Set default style.
    ylabel=''
    if not title: title = 'prova' 
    if not figsize: figsize = [default_figwidth, 8]
    pprint (df_bar.index.names[:-1])
    # Plot 
    mean = df_bar.groupby(level=df_bar.index.names[:-1]).mean().unstack(unstack_level)
    std = df_bar.groupby(level=df_bar.index.names[:-1]).std().unstack(unstack_level)
    axes = mean.groupby(level=groupby_level) \
        .plot(yerr=std, kind='bar', grid=True, width=0.8, rot=rot, figsize=figsize,
              fontsize=default_fontsize, colormap=colormap)
    for ax in axes:
        # Title.
        ax.set_title(title)
        # X label.
        ax.set_xlabel(xlabel)
        # Y axis.
        ax.set_ylabel(ylabel)
plot_performance(dfs_per, performance_metric=['avg_fps','avg_time_ms'])
plot_performance(dfs_per, performance_metric=['graph_load_time_ms','images_load_time_avg_ms'])